In [ ]:
#IMPORTACION DE LAS LIBRERIAS A UTILIZAR

import pandas as pd #librería para analizar datos
import numpy as np #librería para algebra linear
import tensorflow as tf #librería para modelos de deep learning
import matplotlib.pyplot as plt # libreraía para realizar los graficos
import optuna # libreria para optimizar modelo
from sklearn.model_selection import train_test_split #parte de la librería que divide los datos
from sklearn.metrics import roc_curve, auc, roc_auc_score # libreria para calcular las metricas en la curva ROC
from sklearn.metrics import precision_score, recall_score, f1_score # librerias para calcular la precision, el recall y el f1 score
from sklearn.preprocessing import MinMaxScaler #libreria para estandarizacion de datos
from sklearn.preprocessing import StandardScaler #libreria para estandarizacion de datos 
from keras.models import Sequential #importar el inicio de un modelo secuencial
from keras.layers import LSTM, Dense # importa capas densas y de LSTM
from sklearn.metrics import confusion_matrix #parte de la librería para hacer la matriz de confusion
from tensorflow import keras #importa keras que se encarga de la transformacion de los datos
from spicy import stats

# cargamos datos y fijamos valores del modelo
data = pd.read_excel("anomalias2_ajustado2.xlsx")
empresa = 'claro'
empresa1 = 'claro_l'

test_size = 0.3 # longitud del testeo
valor = 1 - test_size
valor1 = valor*400
valor1 = int(valor1) #longitud del la parte de entrenamiento
TIME_STEPS = 30 #valor que se puede variar en el tiempo
epochs = 100 
batch_size = 32
etiqueta = data[empresa1][valor1:]
#PREPROCESAMIENTO DE LOS DATOS

data['valor_normalizado'] = data[empresa]
train, test = train_test_split(data['valor_normalizado'], test_size=test_size, shuffle = False)
train = pd.DataFrame(train)
test= pd.DataFrame(test)
scaler = StandardScaler()
scaler = scaler.fit(train[['valor_normalizado']])
train['valor_normalizado'] = scaler.transform(train[['valor_normalizado']])
test['valor_normalizado'] = scaler.transform(test[['valor_normalizado']])

def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)        
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X_train, y_train = create_dataset(train[['valor_normalizado']], train.valor_normalizado, TIME_STEPS)
X_test, y_test = create_dataset(test[['valor_normalizado']], test.valor_normalizado, TIME_STEPS)

#CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1])),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, X_train, epochs=epochs, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0) #ENTRENAMIENTO
mse = model.evaluate(X_train, X_train, verbose=0)
loss = history.history['loss']
val_loss = history.history['val_loss']

# desprocesamiento de los datos de testeo
a = pd.DataFrame(X_test[0].flat)
b = pd.DataFrame(y_test)
datos_testeo = pd.concat((a,b)).reset_index(drop=True)

#predicciones
predictions = model.predict(X_test)
predicciones1 = pd.DataFrame(predictions[0].flat)
predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
predicciones = pd.concat((predicciones1, predicciones2))
predicciones.reset_index(drop=True, inplace=True)
datos_testeo.reset_index(drop=True, inplace=True)

#calculo del error de reconstrucción
df_test = pd.DataFrame()
df_test['x_test'] = datos_testeo
df_test['predic'] = predicciones
df_test['error'] = df_test['x_test'] - df_test['predic']
df_test['error2'] = df_test['error']**2

#cálculo del threshold
mu = np.mean(df_test['error2'])  # Media
sigma = np.std(df_test['error2'])  # Desviación estándar
alpha = 0.05  # Nivel de significancia del 5%
z_score = stats.norm.ppf(1 - alpha/2)  # Utilizando la función de percentil inverso de la distribución normal
threshold = mu - (z_score * (sigma / np.sqrt(len(df_test['error2'])))) #Umbral gaussiano

anomalias = df_test[df_test['error2'] > threshold] #creación de la variable anomalias que contiene a aquellos errores mayores al umbral

#etiquetas de las anomalias
predic = pd.DataFrame()
predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
predic.loc[anomalias.index + valor1, 'Valor'] = 1
predic = predic[valor1:]
confusion = confusion_matrix(etiqueta, predic)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

precision = precision_score(etiqueta, predic)
recall = recall_score(etiqueta, predic)
f1 = f1_score(etiqueta, predic)
fpr, tpr, thresholds = roc_curve(etiqueta, predic)
roc_auc = auc(fpr, tpr)
fpr, tpr, _ = roc_curve(etiqueta, predic)
roc_auc = roc_auc_score(etiqueta, predic)

print("Verdaderos Positivos (TP):", TP)
print("Verdaderos Negativos (TN):", TN)
print("Falsos Positivos (FP):", FP)
print("Falsos Negativos (FN):", FN)
print(f"Precisión: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print("Área bajo la curva (AUC): {:.2f}".format(roc_auc))

# Gráfico de Pérdida en Entrenamiento y Validación

plt.figure(figsize=(12, 6))
plt.plot(loss, label='Pérdida en Entrenamiento', color='blue')
plt.plot(val_loss, label='Pérdida en Validación', color='red')
plt.legend()
plt.title('Gráfico de Pérdida durante el Entrenamiento')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.grid(True)

# Gráfico de Pérdida y Umbral para Detección de Anomalías

plt.figure(figsize=(12, 6))
plt.plot(df_test.index, df_test.error2, label='Pérdida', color='blue')
plt.axhline(y=threshold, color='red', linestyle='--', label='Umbral')
plt.title('Pérdida vs. Umbral para Detección de Anomalías')
plt.xlabel('Índice de Datos')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

# Gráfico de Datos Originales vs. Predicciones (con Anomalías)

plt.figure(figsize=(12, 6))
plt.plot(datos_testeo, label='Datos Originales', color='blue')
plt.plot(predicciones, label='Predicciones', color='green')
plt.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=50)
plt.title('Datos Originales vs. Predicciones (con Anomalías)')
plt.xlabel('Índice de Datos')
plt.ylabel('Valor Normalizado')
plt.legend()
plt.grid(True)

#CURVA ROC

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
# BUSQUEDA DEL THRESHOLD OPTIMO

y = []
for x in range(0,10000):
    y.append((x/1000))
pre = []
rec = []
f1 = []
optimos =[]
for i in y:
    threshold = i
    anomalias = df_test[df_test['error2'] > threshold]
    predic = pd.DataFrame()
    predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
    predic.loc[anomalias.index+valor1, 'Valor'] = 1
    predic = predic[valor1:]
    precision = precision_score(etiqueta, predic,zero_division=0)
    recall = recall_score(etiqueta, predic,zero_division=0)
    f = f1_score(etiqueta, predic,zero_division=0)
    pre.append(precision)
    rec.append(recall)
    f1.append(f)

plt.figure(figsize=(10, 6))
plt.plot(pre, label='Precision', color='blue')
plt.plot(f1, label='F1 Score', color='green')
plt.plot(rec, label='Recall', color='red')
plt.legend()
plt.xlabel('Umbral')
plt.ylabel('Valor de la Métrica')
plt.title('Métricas de Clasificación en función del Umbral')
plt.grid(True) 
plt.show()

In [ ]:
# Convierte las listas en arrays de numpy para facilitar la manipulación
pre = np.array(pre)
rec = np.array(rec)
f1 = np.array(f1)

# Encuentra el índice del umbral que maximiza F1
indice_umbral_optimo = np.argmax(f1)

# Obtiene el umbral óptimo
umbral_optimo = y[indice_umbral_optimo]

In [ ]:
#### REEMPLAZAR ###
threshold = umbral_optimo
anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD
predic = pd.DataFrame()
predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
predic.loc[anomalias.index + valor1, 'Valor'] = 1
predic = predic[valor1:]

confusion = confusion_matrix(etiqueta, predic)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

precision = precision_score(etiqueta, predic)
recall = recall_score(etiqueta, predic)
f1 = f1_score(etiqueta, predic)
fpr, tpr, thresholds = roc_curve(etiqueta, predic)
roc_auc = auc(fpr, tpr)
fpr, tpr, _ = roc_curve(etiqueta, predic)
roc_auc = roc_auc_score(etiqueta, predic)

print("Verdaderos Positivos (TP):", TP)
print("Verdaderos Negativos (TN):", TN)
print("Falsos Positivos (FP):", FP)
print("Falsos Negativos (FN):", FN)
print(f"Precisión: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print("Área bajo la curva (AUC): {:.2f}".format(roc_auc))

# Gráfico de Pérdida en Entrenamiento y Validación

plt.figure(figsize=(12, 6))
plt.plot(loss, label='Pérdida en Entrenamiento', color='blue')
plt.plot(val_loss, label='Pérdida en Validación', color='red')
plt.legend()
plt.title('Gráfico de Pérdida durante el Entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.grid(True)

# Gráfico de Pérdida y Umbral para Detección de Anomalías

plt.figure(figsize=(12, 6))
plt.plot(df_test.index, df_test.error2, label='Pérdida', color='blue')
plt.axhline(y=threshold, color='red', linestyle='--', label='Umbral')
plt.title('Pérdida vs. Umbral para Detección de Anomalías')
plt.xlabel('Índice de Datos')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

# Gráfico de Datos Originales vs. Predicciones (con Anomalías)

plt.figure(figsize=(12, 6))
plt.plot(datos_testeo, label='Datos Originales', color='blue')
plt.plot(predicciones, label='Predicciones', color='green')
plt.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=50)
plt.title('Datos Originales vs. Predicciones (con Anomalías)')
plt.xlabel('Índice de Datos')
plt.ylabel('Valor Normalizado')
plt.legend()
plt.grid(True)

#CURVA ROC

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos (FPR)')
plt.ylabel('Tasa de Verdaderos Positivos (TPR)')
plt.title('Curva ROC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
def objective(trial):
    # Definir los espacios de búsqueda para los hiperparámetros
    num_units1 = trial.suggest_int('num_units1', 64, 128)
    num_units2 = trial.suggest_int('num_units2', 32, 64)
    num_units3 = trial.suggest_int('num_units3', 8, 16)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    # Crear un modelo Autoencoder
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1])),
        tf.keras.layers.Dense(num_units1, activation='relu'),
        tf.keras.layers.Dense(num_units2, activation='relu'),
        tf.keras.layers.Dense(num_units3, activation='relu'),
        tf.keras.layers.Dense(num_units2, activation='relu'),
        tf.keras.layers.Dense(num_units1, activation='relu'),
        tf.keras.layers.Dense(X_train.shape[1], activation='linear')])
    # Compilar el modelo con los hiperparámetros seleccionados
    model.compile(optimizer='adam', loss='mean_squared_error')  # Corregido

    # Entrenar el modelo con los datos de entrenamiento y prueba
    history = model.fit(X_train, X_train, epochs=100, batch_size=batch_size, validation_data=(X_train, X_train), verbose=0)

    # Calcular la métrica de interés (puede ser la precisión, el AUC, etc.)
    # En este ejemplo, usaremos la pérdida en los datos de prueba
    loss = model.evaluate(X_train, X_train)  # Corregido

    return loss

# Crear el estudio de Optuna y ejecutar la minimizacion de la pérdida
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Obtener los mejores hiperparámetros encontrados
best_params = study.best_params

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros encontrados:", best_params)

In [ ]:
#UTILIZAMOS EL MODELO CON LOS NUEVOS PARÁMETROS, Y BATCHSIZE
#CREACION DEL MODELO AUTOENCODER CON CAPAS DENSAS
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1])),
    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units3'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units2'], activation='relu'),
    tf.keras.layers.Dense(best_params['num_units1'], activation='relu'),
    tf.keras.layers.Dense(X_train.shape[1], activation='linear')])

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, X_train, epochs=50, batch_size=best_params['batch_size'], validation_data=(X_train, X_train), verbose=0)
mse = model.evaluate(X_train, X_train, verbose=0)

a = pd.DataFrame(X_test[0].flat)
b = pd.DataFrame(y_test)
datos_testeo = pd.concat((a,b)).reset_index(drop=True)

predictions = model.predict(X_test)
predicciones1 = pd.DataFrame(predictions[0].flat)
predicciones2 = pd.DataFrame(predictions.flat)[(TIME_STEPS-1)::TIME_STEPS]
predicciones = pd.concat((predicciones1, predicciones2))

predicciones.reset_index(drop=True, inplace=True)
datos_testeo.reset_index(drop=True, inplace=True)

loss = history.history['loss']
val_loss = history.history['val_loss']

df_test = pd.DataFrame()
df_test['x_test'] = datos_testeo
df_test['predic'] = predicciones
df_test['error'] = df_test['x_test'] - df_test['predic']
df_test['error2'] = df_test['error']**2

predic = pd.DataFrame()
predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
predic.loc[anomalias.index + valor1, 'Valor'] = 1
predic = predic[valor1:]
confusion = confusion_matrix(etiqueta, predic)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

precision = precision_score(etiqueta, predic)
recall = recall_score(etiqueta, predic)
f1 = f1_score(etiqueta, predic)
fpr, tpr, thresholds = roc_curve(etiqueta, predic)
roc_auc = auc(fpr, tpr)

print("Verdaderos Positivos (TP):", TP)
print("Verdaderos Negativos (TN):", TN)
print("Falsos Positivos (FP):", FP)
print("Falsos Negativos (FN):", FN)
print(f"Precisión: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print("Área bajo la curva (AUC): {:.2f}".format(roc_auc))

# Gráfico de Pérdida en Entrenamiento y Validación

plt.figure(figsize=(12, 6))
plt.plot(loss, label='Pérdida en Entrenamiento', color='blue')
plt.plot(val_loss, label='Pérdida en Validación', color='red')
plt.legend()
plt.title('Gráfico de Pérdida durante el Entrenamiento')
plt.xlabel('Época')
plt.ylabel('Pérdida')
plt.grid(True)

# Gráfico de Pérdida y Umbral para Detección de Anomalías

plt.figure(figsize=(12, 6))
plt.plot(df_test.index, df_test.error2, label='Pérdida', color='blue')
plt.axhline(y=threshold, color='red', linestyle='--', label='Umbral')
plt.title('Pérdida vs. Umbral para Detección de Anomalías')
plt.xlabel('Índice de Datos')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

# Gráfico de Datos Originales vs. Predicciones (con Anomalías)

plt.figure(figsize=(12, 6))
plt.plot(datos_testeo, label='Datos Originales', color='blue')
plt.plot(predicciones, label='Predicciones', color='green')
plt.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=50)
plt.title('Datos Originales vs. Predicciones (con Anomalías)')
plt.xlabel('Índice de Datos')
plt.ylabel('Valor Normalizado')
plt.legend()
plt.grid(True)


In [ ]:
#MEJORA DEL UMBRAL

y = []
for x in range(0,10000):
    y.append((x/1000))
pre = []
rec = []
f1 = []
optimos =[]
for i in y:
    threshold = i
    anomalias = df_test[df_test['error2'] > threshold]
    predic = pd.DataFrame()
    predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
    predic.loc[anomalias.index+valor1, 'Valor'] = 1
    predic = predic[valor1:]
    precision = precision_score(etiqueta, predic,zero_division=0)
    recall = recall_score(etiqueta, predic,zero_division=0)
    f = f1_score(etiqueta, predic,zero_division=0)
    pre.append(precision)
    rec.append(recall)
    f1.append(f)

#Grafico de umbral
plt.figure(figsize=(10, 6))
plt.plot(pre, label='Precision', color='blue')
plt.plot(f1, label='F1 Score', color='green')
plt.plot(rec, label='Recall', color='red')
plt.legend()
plt.xlabel('Umbral')
plt.ylabel('Valor de la Métrica')
plt.title('Métricas de Clasificación en función del Umbral')
plt.grid(True) 
plt.show()

In [ ]:
# Convierte las listas en arrays de numpy para facilitar la manipulación
pre = np.array(pre)
rec = np.array(rec)
f1 = np.array(f1)
# Encuentra el índice del umbral que maximiza f1
indice_umbral_optimo = np.argmax(f1)
# Obtiene el umbral óptimo
umbral_optimo = y[indice_umbral_optimo]
# Encuentra el índice del umbral que maximiza recall
indice_umbral_optimo = np.argmax(rec)
# Obtiene el umbral óptimo
umbral_optimo1 = y[indice_umbral_optimo]

In [ ]:
#### REEMPLAZAR ###
threshold = umbral_optimo
anomalias = df_test[df_test['error2'] > threshold] #ANOMALIA DE ACUERDO A LO QUE SEA MAYOR AL THRESHOLD

# Gráfico de Pérdida en Entrenamiento y Validación
predic = pd.DataFrame()
predic = pd.DataFrame({'Valor': [0] * len(data[empresa1])})
predic.loc[anomalias.index + valor1, 'Valor'] = 1
predic = predic[valor1:]
confusion = confusion_matrix(etiqueta, predic)

TP = confusion[1, 1]
TN = confusion[0, 0]
FP = confusion[0, 1]
FN = confusion[1, 0]

precision = precision_score(etiqueta, predic)
recall = recall_score(etiqueta, predic)
f1 = f1_score(etiqueta, predic)

print("Verdaderos Positivos (TP):", TP)
print("Verdaderos Negativos (TN):", TN)
print("Falsos Positivos (FP):", FP)
print("Falsos Negativos (FN):", FN)
print(f"Precisión: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Gráfico de perdida y mbral para deteccion de anomalias

plt.figure(figsize=(12, 6))
plt.plot(loss, label='Pérdida en Entrenamiento', color='blue')
plt.plot(val_loss, label='Pérdida en Validación', color='red')
plt.legend()
plt.title('Gráfico de Pérdida durante el Entrenamiento')
plt.xlabel('Epochs')
plt.ylabel('Pérdida')
plt.grid(True)

# Gráfico de Pérdida y Umbral para Detección de Anomalías

plt.figure(figsize=(12, 6))
plt.plot(df_test.index, df_test.error2, label='Pérdida', color='blue')
plt.axhline(y=threshold, color='red', linestyle='--', label='Umbral')
plt.title('Pérdida vs. Umbral para Detección de Anomalías')
plt.xlabel('Índice de Datos')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True)

# Gráfico de Datos Originales vs. Predicciones (con Anomalías)

plt.figure(figsize=(12, 6))
plt.plot(datos_testeo, label='Datos Originales', color='blue')
plt.plot(predicciones, label='Predicciones', color='green')
plt.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=50)
plt.title('Datos Originales vs. Predicciones (con Anomalías)')
plt.xlabel('Índice de Datos')
plt.ylabel('Valor Normalizado')
plt.legend()
plt.grid(True)



In [ ]:
# Crear una figura y un solo eje y
fig, ax = plt.subplots(figsize=(12, 6))

# Gráfico de Datos Originales y Predicciones
ax.plot(datos_testeo, label='Datos Originales', color='blue', linestyle = '--', linewidth = 0.8)
ax.plot(predicciones, label='Predicciones', color='black')
ax.scatter(anomalias.index, anomalias['x_test'], color='red', label='Anomalías', marker='o', s=20)
ax.set_title('Datos de tráfico vs. Predicciones (con Anomalías)', fontweight = 'bold')
ax.set_xlabel('Datos de Testeo')
ax.set_ylabel('Valor Normalizado', color='black')  # Eje izquierdo
ax.legend(loc='upper left')

# Crear un segundo conjunto de etiquetas en el eje y para la pérdida y el umbral
ax2 = ax.twinx()
ax2.plot(df_test.index, df_test.error2, label='Pérdida', color='red', linestyle='--', linewidth = 1.5)
ax2.set_ylabel('Pérdida y Umbral', color='black')  # Eje derecho
ax2.legend(loc='upper right')
y2_min = 0
y2_max = max(df_test.error2)*2

# Establecer los límites en el eje y del segundo conjunto de datos
ax2.set_ylim(y2_min, y2_max)  # Reemplaza y2_min y y2_max con los valores deseados

# Ajustar el espaciado entre los gráficos
plt.tight_layout()

# Mostrar el gráfico combinado
plt.show()
